In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (14, 6)
plt.style.use('ggplot')
from datetime import datetime
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
plt.rcParams['font.family'] = 'sans-serif'
pd.set_option('display.width', 1000)
#pd.set_option('float_format', '{:f}'.format)
#pd.set_option('display.mpl_style', 'default') # Make the graphs a bit prettier

import pandas_profiling

In [ ]:
import imageio
import glob
import natsort
import re
from functools import reduce

<h1> Limpieza </h1>

In [ ]:
molinetes_2014 = pd.read_csv('data/molinetes-2014.csv', sep=';')
molinetes_2015 = pd.read_csv('data/molinetes-2015.csv', sep=',')
molinetes_2016 = pd.read_csv('data/molinetes-2016.csv', sep=';')
molinetes_2017 = pd.read_csv('data/molinetes-2017.csv', sep=';')
molinetes_2018 = pd.read_csv('data/molinetes-2018.csv', sep=',')

In [ ]:
molinetes_2014.drop(['HASTA', 'MOLINETE', 'PAX_PAGO', 'PAX_PASES_PAGOS', 'PAX_FRANQ'], axis=1, inplace=True)
molinetes_2015.drop(['periodo', 'hasta', 'molinete', 'pax_pagos', 'pax_pases_pagos', 'pax_franq'], axis=1, inplace=True)
molinetes_2016.drop(['PERIODO', 'HASTA', 'MOLINETE', 'PAX_PAGOS', 'PAX_PASES_PAGOS', 'PAX_FRANQ'], axis=1, inplace=True)
molinetes_2017.drop(['PERIODO', 'HASTA', 'MOLINETE', 'PAX_PAGOS', 'PAX_PASES_PAGOS', 'PAX_FRANQ', 'ID'], axis=1, inplace=True)
molinetes_2018.drop(['periodo', 'hasta', 'molinete', 'pax_pagos', 'pax_pases_pagos', 'pax_franq'], axis=1, inplace=True)

In [ ]:
molinetes_2014.head()

In [ ]:
dfs = [molinetes_2014, molinetes_2015, molinetes_2016, molinetes_2017, molinetes_2018]
columnas = ['fecha', 'hora', 'linea', 'estacion', 'total']

for df in dfs:
    print(df.columns)
    df.columns = columnas
    df['timestamp'] = df['fecha'] +' '+ df['hora']
    df.loc[(df['linea'] == 'LineaA') | (df['linea'] == 'A'), 'linea'] = 'LINEA_A'
    df.loc[(df['linea'] == 'LineaB') | (df['linea'] == 'B'), 'linea'] = 'LINEA_B'
    df.loc[(df['linea'] == 'LineaC') | (df['linea'] == 'C'), 'linea'] = 'LINEA_C'
    df.loc[(df['linea'] == 'LineaD') | (df['linea'] == 'D'), 'linea'] = 'LINEA_D'
    df.loc[(df['linea'] == 'LineaE') | (df['linea'] == 'E'), 'linea'] = 'LINEA_E'
    df.loc[(df['linea'] == 'LineaH') | (df['linea'] == 'H'), 'linea'] = 'LINEA_H'
    

molinetes_2016.drop(molinetes_2016[molinetes_2016['linea'] == 'TALLER/PRUEBAS'].index, inplace=True) 

In [ ]:
molinetes_2014['timestamp'] = molinetes_2014['timestamp'] + ':00'

In [ ]:
molinetes_2017.head(5)

In [ ]:
molinetes_2017['timestamp'] = molinetes_2017['timestamp'].str.replace('/', '-')
molinetes_2016['timestamp'] = molinetes_2016['timestamp'].str.replace('/', '-')

In [ ]:
molinetes_2014['timestamp'] = pd.to_datetime(molinetes_2014['timestamp'])
molinetes_2015['timestamp'] = pd.to_datetime(molinetes_2015['timestamp'])
molinetes_2016['timestamp'] = pd.to_datetime(molinetes_2016['timestamp'], format='%d-%m-%Y %H:%M:%S')
molinetes_2017['timestamp'] = pd.to_datetime(molinetes_2017['timestamp'], format='%d-%m-%Y %H:%M:%S')
molinetes_2018['timestamp'] = pd.to_datetime(molinetes_2018['timestamp'])

In [ ]:
molinetes_2014.to_csv('data/molinetes-2014-cleaned.csv')
molinetes_2015.to_csv('data/molinetes-2015-cleaned.csv')
molinetes_2016.to_csv('data/molinetes-2016-cleaned.csv')
molinetes_2017.to_csv('data/molinetes-2017-cleaned.csv')
molinetes_2018.to_csv('data/molinetes-2018-cleaned.csv')

<h1> Análisis </h1>

In [ ]:
molinetes_2014 = pd.read_csv('data/molinetes-2014-cleaned.csv', sep=',', parse_dates=['timestamp'], index_col='timestamp',
                            dtype={'linea':'category', 'estacion':'category', 'total':int},
                             usecols=['timestamp', 'fecha', 'hora', 'linea', 'estacion', 'total'])
molinetes_2015 = pd.read_csv('data/molinetes-2015-cleaned.csv', sep=',', parse_dates=['timestamp'], index_col='timestamp',
                             dtype={'linea':'category', 'estacion':'category', 'total':int},
                             usecols=['timestamp', 'fecha', 'hora', 'linea', 'estacion', 'total'])
molinetes_2016 = pd.read_csv('data/molinetes-2016-cleaned.csv', sep=',', parse_dates=['timestamp'], index_col='timestamp',
                             dtype={'linea':'category', 'estacion':'category', 'total':int},
                             usecols=['timestamp', 'fecha', 'hora', 'linea', 'estacion', 'total'])
molinetes_2017 = pd.read_csv('data/molinetes-2017-cleaned.csv', sep=',', parse_dates=['timestamp'], index_col='timestamp',
                             dtype={'linea':'category', 'estacion':'category', 'total':int},
                             usecols=['timestamp', 'fecha', 'hora', 'linea', 'estacion', 'total'])
molinetes_2018 = pd.read_csv('data/molinetes-2018-cleaned.csv', sep=',', parse_dates=['timestamp'], index_col='timestamp', 
                             dtype={'linea':'category', 'estacion':'category', 'total':int},
                             usecols=['timestamp', 'fecha', 'hora', 'linea', 'estacion', 'total'])

dfs = [molinetes_2014, molinetes_2015, molinetes_2016, molinetes_2017, molinetes_2018]

In [ ]:
molinetes_2014.año = 2014
molinetes_2015.año = 2015
molinetes_2016.año = 2016
molinetes_2017.año = 2017
molinetes_2018.año = 2018

In [ ]:
molinetes_2018.head()

In [ ]:
molinetes_2018.dropna(inplace=True)

molinetes_2018['estacion'] = molinetes_2018['estacion'].str.lower()
molinetes_2018['linea'] = molinetes_2018['linea'].str.lower()

In [ ]:
molinetes_2018.loc[(molinetes_2018['estacion'] == 'agãâãâãâãâãâãâãâãâãâãâãâãâãâãâãâãâ¼ero') |
                   (molinetes_2018['estacion'] == 'agãâãâãâãâãâãâãâãâ¼ero') |
                   (molinetes_2018['estacion'] == 'agãâ¼ero') |
                   (molinetes_2018['estacion'] == 'agãâãâ¼ero') |
                   (molinetes_2018['estacion'] == 'agüero') |
                   (molinetes_2018['estacion'] == 'agã¼ero'), 'estacion'] = 'aguero'


In [ ]:
molinetes_2018.loc[(molinetes_2018['estacion'].str.contains('saenz')), 'estacion'] = 'saenz peña'

In [ ]:
molinetes_2018.head()

In [ ]:
df_proms = pd.DataFrame()

df_proms['year'] = [2014, 2015, 2016, 2017, 2018]
df_proms['prom_by_year'] = [df['total'].sum() for df in dfs]
df_proms['prom_by_month'] = [df.groupby(df.index.month)['total'].sum().mean() for df in dfs]
df_proms['prom_by_week'] = [df.groupby(df.index.week)['total'].sum().mean() for df in dfs]
df_proms['prom_by_day'] = [df.groupby(df.index.floor('d'))['total'].sum().mean() for df in dfs]

In [ ]:
for col in df_proms.columns[1:]:
    df_proms.plot.line(x='year', y=col, subplots=True)
    plt.xticks(df_proms['year'].values)
    plt.yticks(df_proms[col].values)
    plt.ticklabel_format(style='plain')
    plt.savefig('images/' + col)

In [ ]:
for idx, df in enumerate(dfs):
    explode = (0.05,0.05,0.05,0.05,0.05,0.05)
    df_to_plot = df.groupby(df['linea'])['total'].sum()#.sort_values(ascending=False)
    df_to_plot.plot.pie(autopct='%1.1f%%', startangle=90, pctdistance=0.85, explode = explode)
    centre_circle = plt.Circle((0,0),0.70,fc='white')
    fig = plt.gcf()
    fig.gca().add_artist(centre_circle)

    plt.title(str(2014 + idx))
    plt.tight_layout()
    plt.savefig('images/lineas' + str(idx))
    plt.show()

In [ ]:
images = []

filenames = [img for img in glob.glob("images/lineas*.png")]

filenames.sort(key=lambda var:[int(x) if x.isdigit() else x for x in re.findall(r'[^0-9]|[0-9]+', var)])


for image_path in filenames:
    images.append(imageio.imread(image_path))
    images.append(imageio.imread(image_path))
    images.append(imageio.imread(image_path))
    images.append(imageio.imread(image_path))
    images.append(imageio.imread(image_path))
    images.append(imageio.imread(image_path))
    
imageio.mimsave('images/lineas_evolucion.gif', images)

In [ ]:
molinetes_2018['weekday'] = molinetes_2018.index.weekday

In [ ]:
molinetes_2018.head()

In [ ]:
week_days = molinetes_2018.groupby([molinetes_2018.index.date, 'weekday'])['total'].sum().groupby('weekday').mean()
week_days.index = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
week_days = week_days[::-1]

In [ ]:
ax = week_days.plot(kind='barh', title='Pasajeros promedio por día')
ax.set_xlabel('Pasajeros')
ax.set_ylabel('Día')
[ax.text(v, i, '{:.0f}'.format(v)) for i, v in enumerate(week_days)];
plt.savefig('images/pasajeros_promedio_por_dia.jpg')

In [ ]:
hours_freq = molinetes_2018.groupby(pd.Grouper(freq='1H'))['total'].sum()
hours_freq = hours_freq.groupby(hours_freq.index.hour).mean()
hours_freq = hours_freq[::-1]

In [ ]:
ax = hours_freq.plot(kind='barh', figsize=(25, 15), title='Pasajeros promedio por hora')
ax.set_ylabel('Hora');
ax.set_xlabel('Pasajeros')
[ax.text(v, i, '{:.0f}'.format(v)) for i, v in enumerate(hours_freq)];
plt.savefig('images/pasajeros_promedio_por_hora.jpg')

In [ ]:
hours_freq_by_station = pd.DataFrame(molinetes_2018.groupby([pd.Grouper(freq='1H'), 'estacion'])['total'].sum())
hours_freq_by_station.reset_index(inplace=True)
hours_freq_by_station.set_index('timestamp', inplace=True)
hours_freq_by_station = hours_freq_by_station.groupby([hours_freq_by_station.index.hour, 'estacion']).mean()
hours_freq_by_station.reset_index(inplace=True)
hours_freq_by_station.set_index('timestamp', inplace=True)

In [ ]:
hours_freq_by_station = hours_freq_by_station[hours_freq_by_station['total'] == hours_freq_by_station.groupby('timestamp')['total'].transform(max)][12:]

In [ ]:
ax = hours_freq_by_station['total'].plot(kind='barh', figsize=(25, 15), title='Estación con más pasajeros promedio por hora')
ax.set_ylabel('Hora')
ax.set_xlabel('Pasajeros')
[ax.text(x=row['total'] + 20, y=idx - 6, s='{:.0f} {}'.format(row['total'], row['estacion']), fontsize=15) for idx, row in hours_freq_by_station.iterrows()];
plt.savefig('images/estaciones_por_hora.jpg')

In [ ]:
print(len(molinetes_2018['estacion'].unique()))

In [ ]:
dolar = pd.read_excel('data/dolar.xls', skiprows=3)
dolar = dolar.loc[:, ['Fecha', 'Tipo de Cambio de Referencia - en Pesos - por Dólar']]

In [ ]:
dolar.rename(columns={'Tipo de Cambio de Referencia - en Pesos - por Dólar':'dolar'}, inplace=True)

In [ ]:
dolar.head()

In [ ]:
dolar_grouped = pd.DataFrame(dolar.set_index('Fecha').groupby(pd.Grouper(freq='M'))['dolar'].mean())
dolar_grouped.reset_index(inplace=True)
dolar_grouped['año'] = [int(x[0]) for x in dolar_grouped['Fecha'].astype(str).str.split('-')];
dolar_grouped['mes_numero'] = [int(x[1]) for x in dolar_grouped['Fecha'].astype(str).str.split('-')];

In [ ]:
precio_boleto = pd.read_csv('data/registro-historico-del-precio-del-boleto.csv')

In [ ]:
molinetes_2014.head()

In [ ]:
df_final = pd.DataFrame(columns=['total', 'año', 'precio', 'dolar', 'mes_numero', 'total_pesos', 'total_dolares'])

for df in dfs:
    month_freq = pd.DataFrame(df.groupby(pd.Grouper(freq='m'))['total'].sum())
    month_freq['año'] = df.año
    month_freq['mes'] = range(1, 13)
    boleto_merge = pd.merge(month_freq, precio_boleto, how='left', left_on=['año', 'mes'], right_on=['año', 'mes_numero'])
    boleto_merge.drop(['mes_numero', 'mes_y'], inplace=True, axis=1)
    final_merge = pd.merge(boleto_merge, 
                       dolar_grouped[['dolar', 'año', 'mes_numero']], 
                       left_on=['mes_x', 'año'], 
                       right_on=['mes_numero', 'año'], 
                       how='left')
    final_merge['total_pesos'] = final_merge['total'] * final_merge['precio']
    final_merge['total_dolares'] = final_merge['total_pesos'] / final_merge['dolar']
    final_merge.drop(['mes_x'], axis=1, inplace=True)
    

    df_final = df_final.append(final_merge, ignore_index=True)

In [ ]:
df_final.loc[57:, 'precio'] = 12.50

In [ ]:
df_final.loc[57:, 'total_pesos'] = df_final.loc[57:, 'total'] * df_final.loc[57:, 'precio']
df_final.loc[57:, 'total_dolares'] = df_final.loc[57:, 'total_pesos'] / df_final.loc[57:, 'dolar']

In [ ]:
df_final.head()

In [ ]:
df_final['año_mes'] = df_final['año'].astype(str) + '_' + df_final['mes_numero'].astype(str)

In [ ]:
df_final['año_mes'].values

In [ ]:
fig, ax = plt.subplots()
df_final[['total_pesos', 'total_dolares', 'año_mes']].plot(ax=ax, xticks=df_final.index, figsize=(30, 15), rot=45, fontsize=18)
tick_idx = plt.xticks()[0]
year_labels = df_final['año_mes'][tick_idx].values
ax.set_xticklabels(year_labels)
plt.ticklabel_format(style='plain', axis='y')
plt.savefig('images/pesos_vs_dolar.jpg')

In [ ]:
df_final_grouped_by_year = df_final.groupby(['año'])['total_pesos', 'total_dolares'].sum()

In [ ]:
df_final_grouped_by_year